In [1]:
import pandas as pd
import warnings

In [2]:
# Suppress UserWarnings emitted by openpyxl due to missing CSS in Hae file
with warnings.catch_warnings(record=True):
    warnings.simplefilter("ignore", category=UserWarning)
    PayPlans = pd.read_excel('../data/paymentRulesMatrix.xlsx', header=11)
MarketingUpdates = pd.read_excel('../data/Marketing Compensation file 2024-999.xlsm')
Mappings = pd.read_excel(io='../data/Marketing Compensation file 2024-999.xlsm', 
                         sheet_name='Lookups', 
                         usecols='E:I', 
                         header=0)
Specialty = pd.read_excel(io='../data/Marketing Compensation file 2024-999.xlsm', 
                         sheet_name='Specialty', 
                         usecols='B:J', 
                         header=2)

In [39]:
OriginalPayPlans = PayPlans.copy()
OriginalPayPlans.set_index('Payment Ruleset Code:Name', inplace=True)
PayPlans['RULENAME'] = PayPlans['Payment Ruleset Code:Name'].str.split(':').str[1]
PayPlans['RuleCode'] = PayPlans['Payment Ruleset Code:Name'].str.split(':').str[0]
# wide_df = PayPlans.pivot_table(index='Date', columns='Variable', values='Value', aggfunc='sum')

In [4]:
ProposeCols = [None] + [col for col in MarketingUpdates.columns if col.lower().startswith('proposal')]
ProposeCol = ProposeCols[6]
# MarketingUpdates.columns

In [31]:
MarketingUpdatesMod = (MarketingUpdates
    .loc[MarketingUpdates['#'].isna()]
    .loc[MarketingUpdates['IT Use.1'].notna()]
    .loc[MarketingUpdates['DONOR COMPENSATION'].notna()]
    .merge(Mappings, how='left', left_on='DONOR COMPENSATION', right_on='Business Name')
    .merge(PayPlans, how='left', left_on='RULENAME', right_on='RULENAME')
    [['IT Use.1', 'Payment Ruleset Code:Name', ProposeCol, 'Business Name', 'RULENAME']]
)

#################################### For Donor App test

In [52]:
col_filter = ['RULENAME', 'RuleCode'] + [col for col in PayPlans.columns if col.lower().startswith('amount')]

PayPlansFil = PayPlans[col_filter]
PayPlansFil.columns = ['RULENAME', 'RuleCode'] + [col[12:] for col in col_filter if col.lower().startswith('amount')]

PayPlansLong = pd.melt(PayPlansFil, id_vars=['RULENAME', 'RuleCode'], var_name='site_code', value_name='amount')
Compensation = PayPlansLong.merge(MarketingUpdatesMod, how='left', left_on=['site_code', 'RULENAME'], right_on=['IT Use.1', 'RULENAME'])

Compensation = Compensation[['site_code', 'RULENAME', 'RuleCode', 'Business Name', 'amount']]
Compensation.columns = ['CenterID', 'ProgramID', 'PaymentNameNL', 'PaymentTypeDescription', 'Amount']
Compensation['ProgramID'] = Compensation['ProgramID'].str.replace('2X ', '')
Compensation['ProgramID'] = Compensation['ProgramID'].str.replace('Week ', '')
Compensation['ProgramID'] = Compensation['ProgramID'].str.replace('3X ', '')
Compensation['ProgramID'] = Compensation['ProgramID'].str.replace('4X ', '')
Compensation['ProgramID'] = Compensation['ProgramID'].str.replace('5X ', '')
Compensation['ProgramID'] = Compensation['ProgramID'].str.replace('6X ', '')
Compensation['ProgramID'] = Compensation['ProgramID'].str.replace('7X ', '')
Compensation['ProgramID'] = Compensation['ProgramID'].str.replace('8X ', '')
Compensation = Compensation[Compensation['Amount'].notna() & (Compensation['Amount'] != '')]
Compensation['CenterID'] = Compensation['CenterID'].str.replace('CP', '')

Compensation.to_excel('../data/Compensation.xlsx', index=False)


#####################################################

In [6]:
# Pivot
MarketingUpdatesMod = MarketingUpdatesMod.pivot_table(index='Payment Ruleset Code:Name', columns='IT Use.1', values=ProposeCol)


In [7]:
SpecialtyMod = (Specialty
    .merge(PayPlans, how='left', left_on='Name (calculated)', right_on='RULENAME')
    [['PROFILECODE', 'Payment Ruleset Code:Name', 'new Fee']]
)
# SpecialtyMod = SpecialtyMod.pivot_table(index='Payment Ruleset Code:Name', columns='PROFILECODE', values = 'new Fee')
# SpecialtyMod.columns = ["Amount ($): " + col for col in SpecialtyMod.columns]


In [11]:
UpdatedPayPlans = PayPlans.copy()
UpdatedPayPlans = pd.DataFrame({
    'Payment Ruleset Code:Name_copy': UpdatedPayPlans['Payment Ruleset Code:Name']
}).join(UpdatedPayPlans)
UpdatedPayPlans.set_index('Payment Ruleset Code:Name', inplace=True)

UpdatedPayPlans.rename(columns={'Payment Ruleset Code:Name_copy': 'Payment Ruleset Code:Name'}, inplace=True)
amount_columns = UpdatedPayPlans.filter(like='Amount ($)').columns
MarketingUpdatesMod[amount_columns] = MarketingUpdatesMod[amount_columns].astype(float)
UpdatedPayPlans = UpdatedPayPlans.drop('RULENAME', axis=1)    
UpdatedPayPlans.update(MarketingUpdatesMod, overwrite=True)
priority_columns = UpdatedPayPlans.filter(like='Priority:').columns
UpdatedPayPlans[priority_columns] = UpdatedPayPlans[priority_columns].astype('Int32')
UpdatedPayPlans.update(SpecialtyMod)

In [12]:
placeholder = 99999
differences = OriginalPayPlans.fillna(placeholder).ne(UpdatedPayPlans.fillna(placeholder))
row_specialty_differences = differences.loc[differences.index.isin(SpecialtyMod.index)]
SpecialtyColumns = SpecialtyMod.columns.intersection(differences.columns)
specialty_differences = row_specialty_differences[SpecialtyColumns]

row_marketing_differences = differences.loc[differences.index.isin(MarketingUpdatesMod.index)]
MarketingColumns = MarketingUpdatesMod.columns.intersection(differences.columns)
marketing_differences = row_marketing_differences[MarketingColumns]


In [13]:
a = UpdatedPayPlans['Amount ($): CP941']['P_TET_PE_STND:P Tetanus Pre-Existing Standard']
b = OriginalPayPlans['Amount ($): CP941']['P_TET_PE_STND:P Tetanus Pre-Existing Standard']
a

nan

In [14]:
from openpyxl import load_workbook
import tempfile
from openpyxl.styles import Font, PatternFill

import io
# import streamlit as st
# uploaded_PayPlan = st.file_uploader("Upload Haemonetics Payment Plan Matrix", type=['xlsx', 'xlsm'])
    
PayPlansNew = UpdatedPayPlans.copy()

def PrepareFile(df, diff1, diff2, file_like_or_path):
    # Save the uploaded file to a temporary file
    if isinstance(file_like_or_path, str):
        tmp_path = file_like_or_path
    else:
        with tempfile.NamedTemporaryFile(delete=False, suffix='.xlsx') as tmp:
            tmp.write(uploaded_PayPlan.getvalue())
            tmp_path = tmp.name

    # Load the workbook and select the active worksheet
    wb = load_workbook(filename=tmp_path)
    ws = wb.active

    # Assuming your data starts from column A and 'df' is your modified DataFrame
    # Iterate over the DataFrame and update the cells starting from row 13 in Excel (index 12 in openpyxl)
    for r in range(df.shape[0]):
        for c in range(df.shape[1]):
            if not pd.isna(df.iloc[r, c]):
                cell = ws.cell(row=r + 13, column=c + 1, value=df.iloc[r, c])
                column_name = df.columns[c]
                row_name = df.index[r]
                if column_name in diff1.columns and row_name in diff1.index:
                    if diff1.loc[row_name, column_name]:
                        cell.fill = PatternFill(start_color='ADD8E6', end_color='ADD8E6', fill_type='solid')
                        cell.font = Font(color='000000')
                if column_name in diff2.columns and row_name in diff2.index:
                    if diff2.loc[row_name, column_name]:
                        cell.fill = PatternFill(start_color='90EE90', end_color='90EE90', fill_type='solid')
                        cell.font = Font(color='000000')

    # Save the modified workbook to another temporary file
    with tempfile.NamedTemporaryFile(delete=False, suffix='.xlsx') as tmp_modified:
        wb.save(tmp_modified.name)
        tmp_modified_path = tmp_modified.name

    # Indicate to the user that the file is ready for download
    # st.session_state['file_ready_for_download'] = True

    return tmp_modified_path

# if uploaded_PayPlan is not None:
#     tmp_modified_file = PrepareFile(PayPlansNew, marketing_differences, specialty_differences, uploaded_PayPlan)

tmp_modified_file = PrepareFile(PayPlansNew, marketing_differences, specialty_differences, '../data/paymentRulesMatrix.xlsx')

In [15]:
import pandas as pd

# Example DataFrame with Int64 column containing <NA>
df = pd.DataFrame({'col1': pd.Series([1, 2, pd.NA, 4], dtype=pd.Int64)})

# Replace <NA> with None
df['col1'] = df['col1'].astype(object).replace(pd.NA, None)

# Optionally, convert back to Int64 if you had converted to object for the replacement
df['col1'] = df['col1'].astype('Int64')

# Display the DataFrame
print(df)

AttributeError: module 'pandas' has no attribute 'Int64'